In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ["HF_HOME"] = "/root/autodl-tmp/.cache/huggingface"

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import pandas as pd
%load_ext autotime
client = QdrantClient(path="./car/")

time: 2.4 s (started: 2025-02-15 10:08:49 +08:00)


In [2]:
import pickle
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter
import pandas as pd
import numpy as np

time: 380 ms (started: 2025-02-15 10:08:52 +08:00)


In [3]:
import pickle
docs = []
for file in os.listdir('./data/'):
    #file not in ['porche.pkl','honda.pkl','bmw.pkl'] and 
    if file.endswith('.pkl'):
        with open(f'./data/{file}', 'rb') as f:
            data = pickle.load(f)

        for md_content in data:
            markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "Header 1"),("##", "Header 2"),("###", "Header 3"),("####", "Header 4")])
            md_header_splits = markdown_splitter.split_text(md_content)
            md_chunks = [", ".join(split.metadata.values()) + '\n' + split.page_content for split in md_header_splits]
            docs.extend(md_chunks)
    # excludes = set('Header 1','CAUTION')
    # md_header_splits = [split for split in md_header_splits if split.metadata['Header 1'] != 'WARNING']

time: 758 ms (started: 2025-02-15 10:08:54 +08:00)


In [4]:
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True) 
title_des_embeds = model.encode(docs,batch_size = 64,max_length = 8192,)['dense_vecs']

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 160/160 [00:02<00:00, 65.17it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 682/682 [01:00<00:00, 11.33it/s]


time: 1min 17s (started: 2025-02-15 10:08:58 +08:00)


In [5]:

# NOTE: consider splitting the data into chunks to avoid hitting the server's payload size limit
# or use `upload_collection` or `upload_points` methods which handle this for you
# WARNING: uploading points one-by-one is not recommended due to requests overhead

client.delete_collection('sales_qa')

client.create_collection(
    collection_name="sales_qa",
        vectors_config = {
        'text': VectorParams(size=1024, distance=Distance.COSINE)  # Adjust vector size & metric
        }
)

client.upsert(
    collection_name="sales_qa",
    points=[
        PointStruct(id=i, vector={"text": tit_des_embed.tolist() },
        payload={'description':doc},
        )
        for i, (tit_des_embed, doc) in enumerate(zip(title_des_embeds, docs))
    ] 
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

time: 29.6 s (started: 2025-02-15 10:10:46 +08:00)
